# Training our Object Detection Model
Srihari Krishnaswamy, 5-24-2023

This colab notebook trains our Yolov5 object detection model using our Roboflow dataset. The new model's weights are based off the Mbari Fathomnet Yolov5 model, but are fine tuned using our data.

Click [here](https://colab.research.google.com/drive/1-95q51PnuaSNUZ4wnVWqrTNg4q382wAz?usp=sharing) to open this notebook in google colab

### Setup
First, we clone the base Yolov5 repository and install the neccesary requirements:

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

### Mounting Google Drive: 
This will come in handy when downloading the newly trained model!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Roboflow Setup
Here, we install the dependency for Roboflow and download the dataset:

In [ ]:
!pip install -q roboflow

In [ ]:
!mkdir datasets
!curl -L "https://app.roboflow.com/ds/kdyV2f3IRA?key=lItXAqxaFx" > datasets/roboflow.zip; cd datasets; unzip roboflow.zip; rm roboflow.zip; cd ..

### Declaring file paths for training:

In [ ]:
BASE_MODEL_PATH = "/content/yolov5/mbari-mb-benthic-33k.pt"  
NEW_MODEL = "model_to_train.pt"
NEW_MODEL_PATH = "/content/yolov5/models/" + NEW_MODEL

### Downloading the base yolov5 model from which we will adjust the weights in training:

In [ ]:
!curl https://zenodo.org/record/5539915/files/mbari-mb-benthic-33k.pt?download=1 -o {BASE_MODEL_PATH}

In [ ]:
!cp {BASE_MODEL_PATH} {NEW_MODEL_PATH}

### Setting up our yaml file with the new class information that we want to train with:

In [ ]:
relative_dataset_path = "../datasets"
yaml_contents = """train: {0}/train/images
val: {0}/valid/images
test: {0}/test/images

nc: 9
names: ['annelida', 'arthropoda', 'cnidaria', 'echinodermata', 'fish', 'mollusca', 'other-invertebrates', 'porifera', 'unidentified-biology']""".format(relative_dataset_path)

yaml_file_name = "ProjectDS.yaml"
yaml_file_path = "yolov5/data/{}".format(yaml_file_name)
!touch {yaml_file_path}
with open(yaml_file_path, 'w') as file:
  file.write(yaml_contents)

!cat {yaml_file_path}

### Kicking off Training! Descriptions of variables available below:

In [ ]:
batch_size = 48 # number of new inputs before weights are re-adjusted
freeze = 18 #layers of the Neural Network that we want to preserve the weights of - the less the layers, the more layers of the model we apply our training algorithm to!
image_size = 640
weights = NEW_MODEL_PATH #the new model after training
data = yaml_file_name #the yaml file with our data
epochs = 12 #the number of 'cycles' we want to train 

# going into the yolov5 directory and training the model!
!cd yolov5; python3 train.py --batch {batch_size} --freeze {freeze} --weights {weights} --data {data} --epochs {epochs} --cache --img {image_size}


### Now download the newly trained model at yolov5/runs/train/exp/last.pt or yolov5/runs/train/exp/best.pt!